In [ ]:
from grid.model.perception.detection.owlv2 import OWLv2
det_model = OWLv2()
obj = "forklift"

humanoidisaacsim_0.run()

In [ ]:
def get_box_center(box, image_size):
    """
    Calculate the center of a single bounding box given in xyxy format.

    Args:
        box (list or np.ndarray): A bounding box with format [x_min, y_min, x_max, y_max].
    
    Returns:
        tuple: Center point (x_center, y_center).
    """
    x_min, y_min, x_max, y_max = box
    if image_size[0] * image_size[1] * .4 < (x_max-x_min) * (y_max-y_min):
        return -1, -1
    print("proportion: ", (((x_max-x_min)*(y_max-y_min))/(image_size[0]*image_size[1])))
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    return x_center, y_center

from grid.utils.types import Velocity

def get_velocity(target, image_size):
    """
    Calculate velocity components based on the target position in the image.

    Args:
        target (list or np.ndarray): Target position in the image (x, y).
        image_size (list or np.ndarray): Size of the image (width, height).
    
    Returns:
        tuple: Linear velocities (x, y) and angular velocity (yaw).
    """
    image_center_x = image_size[0] / 2
    image_center_y = image_size[1] / 2

    # Compute offset from the center of the image
    offset_x = target[0] - image_center_x
    offset_y = target[1] - image_center_y

    # Normalized offset
    norm_offset_x = offset_x / image_center_x
    norm_offset_y = offset_y / image_center_y

    # Velocity calculations
    linear_velocity_x = norm_offset_x  # [-1, 1] range
    linear_velocity_y = norm_offset_y  # [-1, 1] range

    # Angular velocity (yaw)
    angular_velocity = norm_offset_x  # Using x offset for yaw

    # Clamp values to ensure they're within the range [-1, 1]
    linear_velocity_x = 0.8
    linear_velocity_y = 0
    angular_velocity_z = max(min(-angular_velocity, 2), -2)

    linear_velocity = Velocity(linear_velocity_x, linear_velocity_y, 0)
    angular_velocity = Velocity(0, 0, angular_velocity_z)

    return linear_velocity, angular_velocity

In [ ]:
import time
from grid.utils import log
import numpy as np

while True:
    # Get the RGB image from the camera
    rgb = humanoidisaacsim_0.getImage("camera_rgb_0")

    if rgb is not None and rgb.data is not None:
        log("grid/rgb_img", rgb)
        # Detect the object

        data = rgb.data
        boxes, scores, labels = det_model.run(rgbimage=data[:, :, :3], prompt=obj)
        if boxes is not None and len(boxes) > 0:
            print("scores: ", scores)
            i = np.argmax(scores)
            mid_x, mid_y = get_box_center(boxes[i], [data.shape[1], data.shape[0]])
            if mid_x == -1 and mid_y == -1:
                humanoidisaacsim_0.locomotion.moveByVelocity(Velocity(0, 0, 0), Velocity(0, 0, 0))
                break
            linear_vel, angular_vel = get_velocity([mid_x, mid_y], [data.shape[1], data.shape[0]])
        else:
            linear_vel = Velocity(0, 0, 0)
            angular_vel = Velocity(0, 0, 1)
        humanoidisaacsim_0.locomotion.moveByVelocity(linear_vel, angular_vel)
        
    time.sleep(.1)